In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">Wyświetl kod</a>''')

In [2]:
import ipywidgets as widgets
from ipywidgets import Layout

style = {'description_width': 'initial'}

In [3]:
n = widgets.FloatLogSlider(value=10000, base=10, min=1, max=10, step=1, description='Liczba symulacji', style=style)
kolejka = widgets.BoundedIntText(value=1, min=1, max=30, step=1, description='Wybierz kolejkę:', disabled=False, style=style)

display(n)
display(kolejka)

button = widgets.Button(description='Zatwierdź',disabled=False,tooltip='Confirm',icon='check') 
output = widgets.Output()
display(button,output)

FloatLogSlider(value=10000.0, description='Liczba symulacji', max=10.0, min=1.0, step=1.0, style=SliderStyle(d…

BoundedIntText(value=1, description='Wybierz kolejkę:', max=30, min=1, style=DescriptionStyle(description_widt…

Button(description='Zatwierdź', icon='check', style=ButtonStyle(), tooltip='Confirm')

Output()

In [4]:
strzelone_dom = {
    1: 13,    2: 8,    3: 13,    4: 9,    5: 12,    6: 14,    7: 8,    8: 10,    9: 11,    10: 15,
    11: 9,    12: 15,    13: 8,    14: 10,    15: 6,    16: 12,    17: 7,    18: 12,    19: 8,    20: 9,
    21: 12,    22: 10,    23: 12,    24: 10,    25: 8,    26: 16,    27: 8,    28: 0,    29: 0,    30: 0
}

In [5]:
strzelone_wyjazd = {
    1: 9,    2: 15,    3: 10,    4: 14,    5: 13,    6: 11,    7: 14,    8: 6,    9: 8,    10: 10,
    11: 10,    12: 6,    13: 16,    14: 9,    15: 10,    16: 4,    17: 5,    18: 7,    19: 14,    20: 5,
    21: 7,    22: 8,    23: 8,    24: 5,    25: 11,    26: 10,    27: 5,    28: 0,    29: 0,    30: 0
}

In [6]:
mecze_liga = 26*8 + 7

In [7]:
import pandas as pd
import os

def calculate_weighted_stats(w):
    base = r'KolejkiEkstraklasa'
    matchdays = []
    for filename in os.listdir(base):
        if int(filename)<w:
            matchday = pd.read_csv(f"{base}/{filename}", header=None)
            matchdays.append(matchday)
    league = pd.concat(matchdays, ignore_index=True)
    g = open("Statystyki/poissonstats-ekstraklasa", "r", encoding='utf-8')
    clubs = [x.strip().split(",")[0] for x in g]
    
    global home_attack_strength
    global home_defence_strength
    global away_attack_strength
    global away_defence_strength
    home_attack_strength = {}
    home_defence_strength = {}
    away_attack_strength = {}
    away_defence_strength = {}
    
    global avg_strzelone_dom_liga
    global avg_strzelone_wyjazd_liga
    avg_strzelone_dom_liga = sum([k*v/(8*sum(range(1,w))) for k,v in strzelone_dom.items() if k<w])
    avg_strzelone_wyjazd_liga = sum([k*v/(8*sum(range(1,w))) for k,v in strzelone_wyjazd.items() if k<w])
    
    for c in clubs:
        weight = 1
        weights_away = []
        weights_home = []
        has = 0
        hds = 0
        aas = 0
        ads = 0
        for index, row in league.iterrows():
            home = row[0].split('-')[0]
            away = row[0].split('-')[1]
            if c==home:
                has += int(row[1].split(':')[0])*weight
                hds += int(row[1].split(':')[1])*weight
                weights_home.append(weight)
                weight = weight+1
            if c==away:
                aas += int(row[1].split(':')[1])*weight
                ads += int(row[1].split(':')[0])*weight
                weights_away.append(weight)
                weight = weight+1
        home_attack_strength[c] = has / (sum(weights_home)*avg_strzelone_dom_liga)
        home_defence_strength[c] = hds / (sum(weights_home)*avg_strzelone_wyjazd_liga)
        away_attack_strength[c] = aas / (sum(weights_away)*avg_strzelone_wyjazd_liga)
        away_defence_strength[c] = ads / (sum(weights_away)*avg_strzelone_dom_liga)

In [8]:
import numpy as np
from collections import Counter
from IPython.display import clear_output

def display_matchday(a):
    clear_output()
    h = open(f"KolejkiEkstraklasa/{str(a)}", "r", encoding='utf-8')
    
    for x in h:
        home_team = x.strip().split(",")[0].split("-")[0]
        away_team = x.strip().split(",")[0].split("-")[1]
        rng = np.random.default_rng()
        results = []
        home = home_attack_strength[home_team]*away_defence_strength[away_team]*avg_strzelone_dom_liga
        away = home_defence_strength[home_team]*away_attack_strength[away_team]*avg_strzelone_wyjazd_liga
        print(home_team, "vs", away_team)
        print("Siła:", home, away)
        for i in range(0,int(n.value)):
            results.append((rng.poisson(home, 1)[0],rng.poisson(away, 1)[0]))
        print("Prawdopodobieństwo czystego konta gospodarza:", sum(x[1]==0 for x in results)/n.value)
        print("Prawdopodobieństwo czystego konta gościa:", sum(x[0]==0 for x in results)/n.value)
        
        print("Najbardziej prawdopodobne wyniki:\n",Counter(results).most_common(7),"\n\n\n")
    h.close()

In [9]:
def on_confirm_button_clicked(b):
    calculate_weighted_stats(kolejka.value)
    with output:
        display_matchday(kolejka.value)
    
button.on_click(on_confirm_button_clicked)

In [11]:
home = home_attack_strength["Podbeskidzie"]*away_defence_strength["Wisła"]*avg_strzelone_dom_liga
away = home_defence_strength["Podbeskidzie"]*away_attack_strength["Wisła"]*avg_strzelone_wyjazd_liga
results = []
rng = np.random.default_rng()
for i in range(0,int(n.value)):
    results.append((rng.poisson(home, 1)[0],rng.poisson(away, 1)[0]))
Counter(results).most_common(35)

[((0, 2), 1134),
 ((0, 3), 945),
 ((1, 2), 891),
 ((0, 1), 888),
 ((1, 3), 842),
 ((1, 1), 700),
 ((0, 4), 644),
 ((1, 4), 508),
 ((2, 2), 349),
 ((0, 5), 343),
 ((0, 0), 341),
 ((2, 3), 302),
 ((1, 0), 262),
 ((2, 1), 257),
 ((1, 5), 252),
 ((2, 4), 224),
 ((0, 6), 132),
 ((1, 6), 127),
 ((2, 5), 118),
 ((2, 0), 105),
 ((3, 2), 89),
 ((3, 3), 87),
 ((0, 7), 63),
 ((3, 1), 61),
 ((1, 7), 47),
 ((3, 4), 34),
 ((2, 6), 34),
 ((3, 0), 31),
 ((3, 5), 29),
 ((4, 3), 23),
 ((4, 2), 18),
 ((4, 1), 18),
 ((3, 6), 15),
 ((1, 8), 11),
 ((0, 8), 9)]